In [ ]:
# Collect data from facebook

from gasp.pnd.facebook import places_to_shp

searchArea = r'D:\gis\VGI_CLASSFI\CLS_BUILD\lmt\search_radius.shp'
epsg = 3763
outepsg = 3763
outShp = r'D:\gis\VGI_CLASSFI\CLS_BUILD\facedata2.shp'

places_to_shp(searchArea, epsg, outepsg, outShp, onlySearchArea=None)

In [ ]:
from gasp.dsn import dsn_data_collection_by_multibuffer

BFS = {
    "lisbon" : {
        "path" : r'D:\gis\VGI_CLASSFI\CLS_BUILD\lmt\search_radius.shp',
        "epsg" : 3763
    }
}
CON_PARAM = {'HOST' : 'localhost', 'PORT' : '5432', 'USER' : 'postgres', 'PASSWORD' : 'admin', 'DB' : 'face_extraction',
            'TEMPLATE' : 'postgis_template'}

dsn_data_collection_by_multibuffer(
    BFS, r'D:\gis\VGI_CLASSFI\CLS_BUILD', CON_PARAM, 'facebook'
)

In [ ]:
from gasp.web import data_from_get



URL = 'https://api.foursquare.com/v2/venues/search'
data = data_from_get(URL, dict(
    client_id=CLIENT_ID, client_secret=CLIENT_SECRET,
    v='20180323',
    ll='40.7243,-74.0018',
    intent='browse',
    radius='250',
    limit=50
))

In [ ]:
import pandas
from gasp.pnd.mng.fld import listval_to_newcols

dataDf = pandas.DataFrame(data['response']['venues'])

dataDf.drop([
    'contact', 'hasPerk', 'referralId', 'venuePage', 'verified',
    'venueChains', 'id'
], axis=1, inplace=True)

dataDf.rename(columns={'name' : 'name_main'}, inplace=True)

dataDf = listval_to_newcols(dataDf, 'location')

dataDf.drop([
    "labeledLatLngs", "neighborhood", "state", "distance",
    "crossStreet", 'country', 'city', 'cc', 'address' 
], axis=1, inplace=True)

dataDf["formattedAddress"] = dataDf["formattedAddress"].astype(str)

dataDf = listval_to_newcols(dataDf, 'stats')
dataDf = listval_to_newcols(dataDf, 'categories')
dataDf = listval_to_newcols(dataDf, 0)

dataDf.drop([
    "primary", "id", "icon"
], axis=1, inplace=True)

dataDf = listval_to_newcols(dataDf, 'beenHere')

dataDf.drop([
    "marked", "unconfirmedCount", "lastCheckinExpiredAt"
], axis=1, inplace=True)

In [ ]:
print dataDf

In [ ]:
from gasp.fmapi.twitter import search_places
from gasp.gdl.anls.prox.bfs import getBufferParam

searchArea = r'D:\gis\VGI_CLASSFI\CLS_BUILD\lmt\search_radius.shp'

EPSG = 3763

x, y, r = getBufferParam(searchArea, EPSG, outSRS=4326)

d = search_places(y, x, r)

In [ ]:
print d
print d[0]['_json']